***
### LDACS
***

#### Figure 3.1

In [ ]:
# Import of required libraries
from traffic.core import Traffic
import plotly.graph_objects as go

# Import of data
t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/01_sources/SAMAX/02_runway_type/to_28.parquet"
)
subset = t.sample(100)

# Plot generation
fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=subset.data.latitude,
        lon=subset.data.longitude,
        marker=dict(
            color="blue",
            size=3,
        ),
        showlegend=False,
    )
)
fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=9,
        center=dict(lat=47.5, lon=8.5),
    ),
)
fig.show()

#### Figure 3.2 A

In [ ]:
# Import of required libraries
from traffic.core import Traffic
from traffic.data import airports
import numpy as np
import plotly.graph_objects as go

# Import of data
t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/01_sources/SAMAX/02_runway_type/to_28.parquet"
)

# Generation of a subset of 200 departures
subset_large = t.sample(2000)
subset_large.data = subset_large.data.query("callsign.str.startswith('SWR')")
subset = subset_large.sample(200).resample("4s").eval()


# Remove part before the start of the take-off roll
def after_to_roll(flight):
    try:
        t0 = flight.data.query(
            "8.56695<longitude<8.5695 and 47.45659<latitude<47.457"
        ).timestamp.min()
        return flight.after(t0)
    except:
        return


subset = subset.iterate_lazy().pipe(after_to_roll).eval(desc="processing")


# Generation of subset containing datapoints inside radius of 20nm
subset_rad = (
    subset.distance(airports["LSZH"])
    .query("distance < 30")
    .eval(max_workers=30, desc="Filtering")
)


# Function required to draw a circle around a point
def circle_coordinates(
    lat: float, lon: float, radius_km: float, num_points: int = 360
):
    """
    Calculate the coordinates forming a circle of a given diameter around a
    centre coordinate.

    Parameters
    ----------
    lat : float
        Latitude of the center point.
    lon : float
        Longitude of the center point.
    radius_km : float
        Radius of the circle in kilometers.
    num_points : int, optional
        Number of points to generate the circle, by default 360.
    """

    # Some definitions and conversions
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    R = 6371.0
    angles = np.linspace(0, 2 * np.pi, num_points)
    circle_lat = []
    circle_lon = []

    for angle in angles:
        # Calculate the latitude and longitude coordinates and append to lists
        lat_point = np.arcsin(
            np.sin(lat_rad) * np.cos(radius_km / R)
            + np.cos(lat_rad) * np.sin(radius_km / R) * np.cos(angle)
        )
        lon_point = lon_rad + np.arctan2(
            np.sin(angle) * np.sin(radius_km / R) * np.cos(lat_rad),
            np.cos(radius_km / R) - np.sin(lat_rad) * np.sin(lat_point),
        )
        circle_lat.append(np.degrees(lat_point))
        circle_lon.append(np.degrees(lon_point))

    # Return the latitude and longitude coordinates of the circle
    return circle_lat, circle_lon


# Convert 30 nautical miles to kilometers
radius_km = 30 * 1.852

# Generate the coordinates representing the circle
circle_lat, circle_lon = circle_coordinates(
    airports["LSZH"].latitude, airports["LSZH"].longitude, radius_km, 360
)

# Plot generation
fig = go.Figure()

for flight in subset:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data.latitude,
            lon=flight.data.longitude,
            line=dict(
                color="#d62728",
                width=1.5,
            ),
            showlegend=False,
        )
    )

for flight in subset_rad:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data.latitude,
            lon=flight.data.longitude,
            line=dict(
                color="#2ca02c",
                width=1.5,
            ),
            showlegend=False,
        )
    )

fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=circle_lat,
        lon=circle_lon,
        line=dict(color="black"),
        showlegend=False,
    )
)
fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8.3,
        center=dict(
            lat=airports["LSZH"].latitude, lon=airports["LSZH"].longitude
        ),
    ),
)
fig.show()

#### Figure 3.2 B

In [ ]:
# Import of required libraries
from traffic.core import Traffic
import plotly.graph_objects as go

# Import of data
t = Traffic.from_file(
    "/mnt/beegfs/store/MIAR/01_sources/SAMAX/02_runway_type/to_28.parquet"
)


# Generation of subsets containing the parts of trajectories before and after
# take-off roll initiation
def before_to_roll(flight):
    try:
        t0 = flight.data.query(
            "8.56695<longitude<8.5695 and 47.45659<latitude<47.457"
        ).timestamp.min()
        return flight.before(t0)
    except:
        return


def after_to_roll(flight):
    try:
        t0 = flight.data.query(
            "8.56695<longitude<8.5695 and 47.45659<latitude<47.457"
        ).timestamp.min()
        return flight.after(t0)
    except:
        return


t_keep = (
    subset.iterate_lazy()
    .pipe(after_to_roll)
    .eval(desc="processing", max_workers=10)
)

t_remove = (
    subset.iterate_lazy()
    .pipe(before_to_roll)
    .eval(desc="processing", max_workers=10)
)

# Plot generation
fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=t_keep.data.latitude,
        lon=t_keep.data.longitude,
        marker=dict(
            color="#2ca02c",
            size=3,
        ),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=t_remove.data.latitude,
        lon=t_remove.data.longitude,
        marker=dict(
            color="#d62728",
            size=3,
        ),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=[47.45659, 47.457, 47.457, 47.45659, 47.45659],
        lon=[8.56695, 8.56695, 8.56951, 8.56951, 8.56695],
        line=dict(
            color="black",
            width=3,
        ),
        showlegend=False,
    )
)
fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=9,
        center=dict(lat=47.5, lon=8.5),
    ),
)
fig.show()

#### Table 3.1

In [ ]:
# Import of required libraries
import pandas as pd

# Import of data
df = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_mass.parquet")

# Print overview per airline
for icao in df["ICAO"].unique():
    print(f"Airline: {icao}")
    df_temp = df.query(f"ICAO == '{icao}'")
    print(f"Number of flights: {len(df_temp)}")
    print(f"Types: {sorted(df_temp.typecode.unique().tolist())} \n")
print(f"Total number of flights: {len(df)}")

#### Figure 3.3

In [ ]:
# Import of required libraries
import pandas as pd
import plotly.express as px

# Import of data
df = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_mass.parquet")

# Generation of bar plot
typecode_counts = df["typecode"].value_counts()
fig = px.bar(
    typecode_counts,
    x=typecode_counts.index,
    y=typecode_counts.values,
    orientation="v",
    text=typecode_counts.values,
)

# Adjust count labels
fig.update_traces(
    texttemplate="%{text}", textposition="outside", textfont=dict(size=16)
)

# Adjust overall layout
fig.update_layout(
    xaxis_title="<b>Aircraft Typecode</b>",
    yaxis_title="<b>Number of flights in dataset</b>",
    yaxis={"categoryorder": "total ascending", "title_standoff": 30},
    xaxis={"side": "bottom", "title_standoff": 30},
    width=1500,
    height=600,
    xaxis_title_font=dict(size=25),
    yaxis_title_font=dict(size=25),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),
    margin=dict(l=0, r=0, t=0, b=0),
)

# Show plot
fig.show()

***
### Meteo
***

#### Figure 3.4

In [ ]:
# Import of required libraries
import plotly.graph_objects as go

# Generation of map
fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=[],
        lon=[],
        name="GERSA SID",
        line=dict(width=5),
        opacity=0.7,
    )
)
fig.update_layout(
    width=2000,
    height=1400,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=13,
        center=dict(lat=47.45954, lon=8.55372),
    ),
)

fig.show()

#### Figure 3.5

In [ ]:
# Import of required libraries
import pandas as pd
import plotly.subplots as sp
import plotly.graph_objs as go

# Import data and generate subset
df_wind = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_wind.parquet")
df_sub = df_wind.iloc[134800:136000]

# Generate plot
fig = sp.make_subplots(
    rows=5,
    cols=1,
    subplot_titles=(
        "<b>Polar representation</b>",
        "",
        "",
        "<b>Cartesian representation</b>",
    ),
    vertical_spacing=0.09,
    row_heights=[1, 1, 0, 1, 1],
)

# Add wind speed trace (raw and 2min average)
fig.add_trace(
    go.Scatter(
        x=df_sub["timestamp"],
        y=df_sub["wind_speed_gnd"],
        mode="lines",
        line=dict(color="#636efa"),
        name="raw measurements",
        showlegend=True,
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_sub["timestamp"],
        y=df_sub["wind_speed_gnd_2min_avg"],
        mode="lines",
        line=dict(color="#ef553b"),
        name="2min average",
        showlegend=True,
    ),
    row=1,
    col=1,
)
fig.update_yaxes(title_text="wind<br>speed<br>[kts]", row=1, col=1)

# Add wind direction trace (raw and 2min average)
fig.add_trace(
    go.Scatter(
        x=df_sub["timestamp"],
        y=df_sub["wind_direction_gnd"],
        mode="lines",
        line=dict(color="#636efa"),
        showlegend=False,
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_sub["timestamp"],
        y=df_sub["wind_direction_gnd_2min_avg"],
        mode="lines",
        line=dict(color="#ef553b"),
        showlegend=False,
    ),
    row=2,
    col=1,
)
fig.update_yaxes(title_text="wind<br>direction<br>[°]", row=2, col=1)

# Add X component of the wind
fig.add_trace(
    go.Scatter(
        x=df_sub["timestamp"],
        y=df_sub["wind_x_2min_avg"],
        mode="lines",
        line=dict(color="#00cc96"),
        name="computed x component",
        showlegend=True,
    ),
    row=4,
    col=1,
)
fig.update_yaxes(title_text="wind<br>speed<br>[kts]", row=4, col=1)

# Add Y component of the wind
fig.add_trace(
    go.Scatter(
        x=df_sub["timestamp"],
        y=df_sub["wind_y_2min_avg"],
        mode="lines",
        name="computed y component",
        line=dict(color="#bd8df9"),
        showlegend=True,
    ),
    row=5,
    col=1,
)
fig.update_yaxes(title_text="wind<br>speed<br>[kts]", row=5, col=1)

# Global x-axis
fig.update_xaxes(title_text="timestamp", row=5, col=1)

# Adjustments for readability
fig.update_xaxes(
    title_font=dict(size=20),
    tickfont=dict(size=16),
)
fig.update_yaxes(
    title_font=dict(size=20),
    tickfont=dict(size=16),
)
fig.update_layout(
    width=1500,
    height=800,
    margin=dict(l=20, r=20, t=70, b=10),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(size=20),
    ),
)
for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=22)

fig.show()

***
### Flightplan
***

#### Figure 3.6

In [ ]:
# Import of required libraries
import plotly.graph_objs as go
from traffic.data import navaids


# Function to generate latitude and longitude lists
def generate_latlon_list(navaid_list):
    lat = []
    lon = []
    for navaid in navaid_list:
        if type(navaid) == tuple:
            lat.append(navaid[0])
            lon.append(navaid[1])
        else:
            dat = navaids.extent("Switzerland").get(navaid)
            lat.append(dat.latitude)
            lon.append(dat.longitude)
    return lat, lon


# Generation of latitude and longitude lists for the different SIDs
gersa_lat, gersa_lon = generate_latlon_list(
    [(47.45892, 8.53771), (47.46233, 8.48958), "BREGO", "ARTAG", "GERSA"]
)
deges_lat, deges_lon = generate_latlon_list(
    [
        (47.45892, 8.53771),
        (47.46233, 8.48958),
        "ZH552",
        (47.40351, 8.39876),
        (47.4128, 8.4559),
        "KLO",
        "KOLUL",
        "DEGES",
    ]
)
vebit_lat, vebit_lon = generate_latlon_list(
    [(47.45892, 8.53771), (47.46233, 8.48958), "BREGO", "VEBIT"]
)
zue_lat, zue_lon = generate_latlon_list(
    [
        (47.45892, 8.53771),
        (47.46233, 8.48958),
        "ZH552",
        (47.40351, 8.39876),
        (47.4128, 8.4559),
        "KLO",
        "ZUE",
    ]
)

# Generate plot
fig = go.Figure()

# Add GERSA SID
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=gersa_lat,
        lon=gersa_lon,
        name="GERSA SID",
        line=dict(width=10),
        opacity=0.7,
    )
)

# Add DEGES SID
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=deges_lat,
        lon=deges_lon,
        name="DEGES SID",
        line=dict(width=10),
        opacity=0.7,
    )
)

# Add VEBIT SID
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=vebit_lat,
        lon=vebit_lon,
        name="VEBIT SID",
        line=dict(width=10),
        opacity=0.7,
    )
)

# Add ZUE SID
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=zue_lat,
        lon=zue_lon,
        name="ZUE SID",
        line=dict(width=10),
        opacity=0.7,
    )
)

# Layout of plot
fig.update_layout(
    width=2000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    legend=dict(font=dict(size=20)),
    mapbox=dict(
        style="carto-positron",
        zoom=9,
        center=dict(lat=47.5, lon=8.5),
    ),
)

fig.show()

#### Figure 3.7

In [ ]:
# Import of required libraries
import pandas as pd
import plotly.graph_objects as go
from geographiclib.geodesic import Geodesic
from traffic.core import Traffic

# Import of trajectories
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/inputs/samax_trajs.parquet")

# Selection of one trajectory
flight = t["SWR45LP_353918"]


# Determine bearing from first to last point of the trajectory
def get_bearing(lat1, lat2, long1, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)["azi1"]
    if brng < 0:
        brng = brng + 360
    return brng


bearing = get_bearing(
    flight.data.latitude.iloc[0],
    flight.data.latitude.iloc[-1],
    flight.data.longitude.iloc[0],
    flight.data.longitude.iloc[-1],
)

In [ ]:
# Create Plot
fig = go.Figure()

# Calculate the midpoint of the bearing line
mid_lat = (flight.data.latitude.iloc[0] + flight.data.latitude.iloc[-1]) / 2
mid_lon = (flight.data.longitude.iloc[0] + flight.data.longitude.iloc[-1]) / 2

# Plot the actual flight trajectory with start and stop markers
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=flight.data.latitude,
        lon=flight.data.longitude,
        line=dict(width=4, color="#6e77f4"),
        name="Flight trajectory",
        showlegend=True,
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=[flight.data.latitude.iloc[0]],
        lon=[flight.data.longitude.iloc[0]],
        marker=dict(size=10, color="#6e77f4"),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=[flight.data.latitude.iloc[-1]],
        lon=[flight.data.longitude.iloc[-1]],
        marker=dict(size=10, color="#6e77f4"),
        showlegend=False,
    )
)
fig.add_annotation(
    x=0.97,
    y=0.76,
    xref="paper",
    yref="paper",
    text="<b>start</b>",
    font={"color": "#6e77f4", "size": 20},
    showarrow=False,
    yshift=10,
)
fig.add_annotation(
    x=0.03,
    y=0.28,
    xref="paper",
    yref="paper",
    text="<b>stop</b>",
    font={"color": "#6e77f4", "size": 20},
    showarrow=False,
    yshift=10,
)

# Plot the straight bearing line
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=[flight.data.latitude.iloc[0], flight.data.latitude.iloc[-1]],
        lon=[
            flight.data.longitude.iloc[0],
            flight.data.longitude.iloc[-1],
        ],
        line=dict(
            width=1,
            color="red",
        ),
        name="Bearing",
        showlegend=True,
    )
)

# Add annotation for bearing at the midpoint of the line
fig.add_annotation(
    x=0.5,
    y=0.51,
    xref="paper",
    yref="paper",
    text=f"<b>{round(bearing,2)}°</b>",
    textangle=bearing - 270,
    font={"color": "red", "size": 20},
    showarrow=False,
    yshift=10,
)

# Layout of plot
fig.update_layout(
    width=1300,
    height=700,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=10.5,
        center=dict(
            lat=mid_lat,
            lon=mid_lon,
        ),
    ),
    legend=dict(font=dict(size=20)),
)
fig.show()

#### Figure 3.8

In [ ]:
# Import of required libraries
import pandas as pd
import plotly.express as px

# Import of SID classification
df_sid = pd.read_parquet(
    "/mnt/beegfs/store/krum/MT/inputs/df_sid.parquet"
).sample(250000)

# Plot historgram of bearings
fig = px.histogram(df_sid.bearing, nbins=180)

# Add vertical lines for the sectors
fig.add_vline(x=75, line_color="red")
fig.add_vline(x=140, line_color="red")
fig.add_vline(x=210, line_color="red")

# Add annotations for the sectors (SIDs)
# ZUE
fig.add_annotation(
    x=37.5,
    y=0.9,
    yref="paper",
    text="ZUE",
    showarrow=False,
    font_size=16,
    font_color="red",
    align="center",
)

# DEGES
fig.add_annotation(
    x=107.5,
    y=0.9,
    yref="paper",
    text="DEGES",
    showarrow=False,
    font_size=16,
    font_color="red",
    align="center",
)

# GERSA
fig.add_annotation(
    x=175,
    y=0.9,
    yref="paper",
    text="GERSA",
    showarrow=False,
    font_size=16,
    font_color="red",
    align="center",
)

# VEBIT
fig.add_annotation(
    x=285,
    y=0.9,
    yref="paper",
    text="VEBIT",
    showarrow=False,
    font_size=16,
    font_color="red",
    align="center",
)

fig.update_layout(
    width=1200,
    height=500,
    showlegend=False,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis_tickfont=dict(size=15),
    yaxis_tickfont=dict(size=15),
    yaxis={"title_standoff": 30},
    xaxis={"title_standoff": 30},
)
fig.update_xaxes(title_text="Departure bearing [°]")
fig.show()

#### Figure 3.9

In [ ]:
# Import of required libraries
import pandas as pd
import plotly.graph_objects as go
from traffic.core import Traffic

# Import of trajectories
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/inputs/samax_trajs.parquet")

# Import of SID classification
df_sid = pd.read_parquet("/mnt/beegfs/store/krum/MT/inputs/df_sid.parquet")

# Reduce to Swiss flights to remove VFR traffic
df_sid = df_sid.query("flight_id.str.startswith('SWR')")

# Generate traffics with 50 flights per SID
subset_VEBIT = (
    df_sid[df_sid["SID"] == "VEBIT"].sample(50)["flight_id"].to_list()
)
t_VEBIT = t[subset_VEBIT]

subset_GERSA = (
    df_sid[df_sid["SID"] == "GERSA"].sample(50)["flight_id"].to_list()
)
t_GERSA = t[subset_GERSA]

subset_DEGES = (
    df_sid[df_sid["SID"] == "DEGES"].sample(50)["flight_id"].to_list()
)
t_DEGES = t[subset_DEGES]

subset_ZUE = df_sid[df_sid["SID"] == "ZUE"].sample(50)["flight_id"].to_list()
t_ZUE = t[subset_ZUE]

# Create Plot
fig = go.Figure()
# GERSA
for flight in t_GERSA[0:49]:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data.latitude,
            lon=flight.data.longitude,
            line=dict(
                color="#6e77f4",
                width=1,
            ),
            showlegend=False,
        )
    )
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=t_GERSA[49].data.latitude,
        lon=t_GERSA[49].data.longitude,
        line=dict(
            color="#6e77f4",
            width=1,
        ),
        name="GERSA",
        showlegend=True,
    )
)

# DEGES
for flight in t_DEGES[0:49]:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data.latitude,
            lon=flight.data.longitude,
            line=dict(
                color="#dd6044",
                width=1,
            ),
            showlegend=False,
        )
    )
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=t_DEGES[49].data.latitude,
        lon=t_DEGES[49].data.longitude,
        line=dict(
            color="#dd6044",
            width=1,
        ),
        name="DEGES",
        showlegend=True,
    )
)

# VEBIT
for flight in t_VEBIT[0:49]:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data.latitude,
            lon=flight.data.longitude,
            line=dict(
                color="#5dc999",
                width=1,
            ),
            showlegend=False,
        )
    )
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=t_VEBIT[49].data.latitude,
        lon=t_VEBIT[49].data.longitude,
        line=dict(
            color="#5dc999",
            width=1,
        ),
        name="VEBIT",
        showlegend=True,
    )
)

# ZUE
for flight in t_ZUE[0:49]:
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data.latitude,
            lon=flight.data.longitude,
            line=dict(
                color="#a167f4",
                width=1,
            ),
            showlegend=False,
        )
    )
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=t_ZUE[49].data.latitude,
        lon=t_ZUE[49].data.longitude,
        line=dict(
            color="#a167f4",
            width=1,
        ),
        name="ZUE",
        showlegend=True,
    )
)

fig.update_layout(
    width=1300,
    height=700,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=9,
        center=dict(lat=47.45928637316676, lon=8.553560073903695),
    ),
    legend=dict(font=dict(size=20)),
)
fig.show()

### Generation of training validation and test data

In [ ]:
# Import of required libraries
import plotly.graph_objects as go
from traffic.core import Traffic

# Import of data
t = Traffic.from_file(
    "/mnt/beegfs/store/krum/MT/encoded_scaled_split/t_val.parquet"
)
sample = t.data.sample(10000)

# Generation of subsets for each month
month_1 = sample.query("month == 1")
month_2 = sample.query("month == 2")
month_3 = sample.query("month == 3")
month_4 = sample.query("month == 4")
month_5 = sample.query("month == 5")
month_6 = sample.query("month == 6")
month_7 = sample.query("month == 7")
month_8 = sample.query("month == 8")
month_9 = sample.query("month == 9")
month_10 = sample.query("month == 10")
month_11 = sample.query("month == 11")
month_12 = sample.query("month == 12")

# Plot one sample per month (x = month sin, y = month cos)
fig = go.Figure()
for i in range(1, 13):
    temp = sample.query(f"month == {i}").iloc[0]
    fig.add_trace(
        go.Scatter(
            x=[temp["month_sin"]],
            y=[temp["month_cos"]],
            mode="markers+text",
            text=[f"<b>{i}</b>"],
            textposition="top center",
            name=f"Month {i}",
            showlegend=False,
            marker=dict(size=9, color="black"),
            textfont=dict(
                size=30,
                color="black",
            ),
        )
    )

# Changes to make the plot look nice
fig.update_layout(
    width=800,
    height=800,
    xaxis=dict(
        title="Month sin",
        tickfont=dict(
            size=30,
            color="black",
        ),
        title_font=dict(
            size=35,
            color="black",
        ),
    ),
    yaxis=dict(
        title="Month cos",
        tickfont=dict(
            size=30,
            color="black",
        ),
        title_font=dict(
            size=35,
            color="black",
        ),
        title_standoff=50,
    ),
)
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()